## Governance Domain: Mortgage Percentage among Homeowners Metric Calculation
* % of homeowners with a mortgage

In [21]:
import pandas as pd
import os
import sys
import numpy as np

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/governance/personal_preparedness/american_community_survey/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'ACSST5Y2022.S2506-Column-Metadata.csv'


c:\Users\jespi\eagle\carb-climate-index-7\scripts\utils\file_helpers.py:78: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,134,135) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'ACSST5Y2022.S2506-Data.csv'
Saved DataFrame as 'by_county.csv'


In [7]:
homeowners_insurance_data_county = pd.read_csv('by_county.csv')
print(len(homeowners_insurance_data_county))
homeowners_insurance_data_county = homeowners_insurance_data_county.rename(columns={'Label (Grouping)':'index'})
homeowners_insurance_data_county = homeowners_insurance_data_county[1:]
homeowners_insurance_data_county.head(5)

# os.remove('by_county.csv')

160


,index,"Alameda County, California!!Estimate","Alameda County, California!!Margin of Error","Alameda County, California!!Percent","Alameda County, California!!Percent Margin of Error","Butte County, California!!Estimate","Butte County, California!!Margin of Error","Butte County, California!!Percent","Butte County, California!!Percent Margin of Error","Contra Costa County, California!!Estimate",...,"Ventura County, California!!Percent","Ventura County, California!!Percent Margin of Error","Yolo County, California!!Estimate","Yolo County, California!!Margin of Error","Yolo County, California!!Percent","Yolo County, California!!Percent Margin of Error","Yuba County, California!!Estimate","Yuba County, California!!Margin of Error","Yuba County, California!!Percent","Yuba County, California!!Percent Margin of Error"
1,Total housing units,"629,159","±1,950","629,159",(X),"90,309",±546,"90,309",(X),"426,327",...,"294,645",(X),"81,252",±635,"81,252",(X),"29,973",±234,"29,973",(X)
2,Occupied housing units,"589,180","±4,004",93.6%,±0.5,"81,353","±1,910",90.1%,±2.0,"411,560",...,94.7%,±0.9,"76,844","±1,623",94.6%,±1.7,"27,942",±908,93.2%,±2.9
3,Vacant housing units,"39,979","±3,194",6.4%,±0.5,"8,956","±1,847",9.9%,±2.0,"14,767",...,5.3%,±0.9,"4,408","±1,360",5.4%,±1.7,"2,031",±882,6.8%,±2.9
4,Homeowner vacancy rate,0.7,±0.3,(X),(X),1.4,±0.8,(X),(X),0.8,...,(X),(X),0.0,±0.5,(X),(X),0.3,±0.5,(X),(X)
5,Rental vacancy rate,6.1,±0.9,(X),(X),4.4,±2.0,(X),(X),1.9,...,(X),(X),1.5,±1.3,(X),(X),2.0,±2.8,(X),(X)


In [8]:
# Transpose the DataFrame
homeowners_insurance_data_county_transpose = homeowners_insurance_data_county.set_index('index').T.reset_index()

homeowners_insurance_data_county_transpose

index,index,Total housing units,Occupied housing units,Vacant housing units,Homeowner vacancy rate,Rental vacancy rate,UNITS IN STRUCTURE,Total housing units,"1-unit, detached","1-unit, attached",...,No rent paid,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI),Occupied units paying rent (excluding units where GRAPI cannot be computed),Less than 15.0 percent,15.0 to 19.9 percent,20.0 to 24.9 percent,25.0 to 29.9 percent,30.0 to 34.9 percent,35.0 percent or more,Not computed
0,"Alameda County, California!!Estimate","629,159","589,180","39,979",0.7,6.1,NaN,"629,159","322,776","56,494",...,"7,107",NaN,"256,368","31,030","33,291","32,166","29,095","24,710","106,076","10,942"
1,"Alameda County, California!!Margin of Error","±1,950","±4,004","±3,194",±0.3,±0.9,NaN,"±1,950","±5,587","±3,099",...,"±1,108",NaN,"±6,109","±2,659","±2,904","±2,634","±2,962","±3,414","±5,453","±1,524"
2,"Alameda County, California!!Percent","629,159",93.6%,6.4%,(X),(X),NaN,"629,159",51.3%,9.0%,...,(X),NaN,"256,368",12.1%,13.0%,12.5%,11.3%,9.6%,41.4%,(X)
3,"Alameda County, California!!Percent Margin of ...",(X),±0.5,±0.5,(X),(X),NaN,(X),±0.9,±0.5,...,(X),NaN,(X),±1.0,±1.1,±1.0,±1.1,±1.3,±1.9,(X)
4,"Butte County, California!!Estimate","90,309","81,353","8,956",1.4,4.4,NaN,"90,309","55,771","2,372",...,"1,445",NaN,"31,556","2,788","2,808","4,164","4,304","1,951","15,541","2,254"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,"Yolo County, California!!Percent Margin of Error",(X),±1.7,±1.7,(X),(X),NaN,(X),±2.7,±1.2,...,(X),NaN,(X),±2.2,±3.5,±3.3,±3.8,±2.5,±5.7,(X)
164,"Yuba County, California!!Estimate","29,973","27,942","2,031",0.3,2.0,NaN,"29,973","19,634","2,001",...,N,NaN,"10,480",525,"1,985","1,302",776,"1,715","4,177","1,141"
165,"Yuba County, California!!Margin of Error",±234,±908,±882,±0.5,±2.8,NaN,±234,"±1,505",±828,...,N,NaN,"±1,540",±390,±903,±690,±423,±917,±978,±885
166,"Yuba County, California!!Percent","29,973",93.2%,6.8%,(X),(X),NaN,"29,973",65.5%,6.7%,...,(X),NaN,"10,480",5.0%,18.9%,12.4%,7.4%,16.4%,39.9%,(X)


In [9]:
homeowners_insurance_data_county_transpose.columns

Index(['index', '    Total housing units', '        Occupied housing units',
       '        Vacant housing units', '        Homeowner vacancy rate',
       '        Rental vacancy rate', 'UNITS IN STRUCTURE',
       '    Total housing units', '        1-unit, detached',
       '        1-unit, attached',
       ...
       '        No rent paid',
       'GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)',
       '    Occupied units paying rent (excluding units where GRAPI cannot be computed)',
       '        Less than 15.0 percent', '        15.0 to 19.9 percent',
       '        20.0 to 24.9 percent', '        25.0 to 29.9 percent',
       '        30.0 to 34.9 percent', '        35.0 percent or more',
       '        Not computed'],
      dtype='object', name='index', length=160)

In [10]:
# retain one of the columns after transposing
# really strange how the names change to multiple '\xa0' before the name after transposing 
columns = [
    'index',
    '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Housing units with a mortgage']
# Select the specified columns
homeowners_insurance_county_columns = homeowners_insurance_data_county_transpose[columns]
homeowners_insurance_county_columns = homeowners_insurance_county_columns.rename(columns={'MORTGAGE STATUS':'mortgage_status', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Housing units with a mortgage':'housing_units_with_mortgage', 'index':'county'})
# Display the selected columns
homeowners_insurance_county_columns.head()

index,county,housing_units_with_morgage
0,"Alameda County, California!!Estimate","221,103"
1,"Alameda County, California!!Margin of Error","±5,945"
2,"Alameda County, California!!Percent",68.7%
3,"Alameda County, California!!Percent Margin of ...",±1.2
4,"Butte County, California!!Estimate","26,714"


In [11]:
homeowners_insurance_county_columns.columns

Index(['county', 'housing_units_with_morgage'], dtype='object', name='index')

In [12]:
homeowners_insurance_county_columns

index,county,housing_units_with_morgage
0,"Alameda County, California!!Estimate","221,103"
1,"Alameda County, California!!Margin of Error","±5,945"
2,"Alameda County, California!!Percent",68.7%
3,"Alameda County, California!!Percent Margin of ...",±1.2
4,"Butte County, California!!Estimate","26,714"
...,...,...
163,"Yolo County, California!!Percent Margin of Error",±3.1
164,"Yuba County, California!!Estimate","11,009"
165,"Yuba County, California!!Margin of Error","±1,442"
166,"Yuba County, California!!Percent",67.5%


In [13]:
# get rid of extra, non-percentage data entries
filtered_county_mortgage = homeowners_insurance_county_columns[~homeowners_insurance_county_columns['county'].str.contains('Margin|Estimate')]

print(len(filtered_county_mortgage))
filtered_county_mortgage.head()

42


index,county,housing_units_with_morgage
2,"Alameda County, California!!Percent",68.7%
6,"Butte County, California!!Percent",56.2%
10,"Contra Costa County, California!!Percent",72.2%
14,"El Dorado County, California!!Percent",68.2%
18,"Fresno County, California!!Percent",65.4%


In [14]:
# Filter and remove " County, California!!Percent" from 'county' column to isolate county name
filtered_county_mortgage = filtered_county_mortgage[~filtered_county_mortgage['county'].str.contains('Margin')]
filtered_county_mortgage['county'] = filtered_county_mortgage['county'].str.replace(' County, California!!Percent', '')

print(len(filtered_county_mortgage))
filtered_county_mortgage.head(50)

42


index,county,housing_units_with_morgage
2,Alameda,68.7%
6,Butte,56.2%
10,Contra Costa,72.2%
14,El Dorado,68.2%
18,Fresno,65.4%
22,Humboldt,57.2%
26,Imperial,61.6%
30,Kern,67.1%
34,Kings,66.2%
38,Lake,52.6%


In [17]:
# pull census tract data for merging
county_tract = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_county_tract = pd.read_csv(county_tract)
ca_county_tract = ca_county_tract.rename(columns={'TRACT': 'census_tract'})
ca_county_tract = ca_county_tract.rename(columns={'County': 'county'})
ca_county_tract = ca_county_tract.drop(columns={'Unnamed: 0','COUNTYFP'})

ca_county_tract

,census_tract,county
0,6085504321,Santa Clara
1,6085504410,Santa Clara
2,6085507003,Santa Clara
3,6085507004,Santa Clara
4,6085502204,Santa Clara
...,...,...
9124,6059001303,Orange
9125,6059001304,Orange
9126,6059001401,Orange
9127,6013367200,Contra Costa


In [19]:
# merge data with tract data
merged_homeowners = pd.merge(ca_county_tract, filtered_county_mortgage, on='county', how='left')
merged_homeowners

,census_tract,county,housing_units_with_morgage
0,6085504321,Santa Clara,67.5%
1,6085504410,Santa Clara,67.5%
2,6085507003,Santa Clara,67.5%
3,6085507004,Santa Clara,67.5%
4,6085502204,Santa Clara,67.5%
...,...,...,...
9124,6059001303,Orange,67.0%
9125,6059001304,Orange,67.0%
9126,6059001401,Orange,67.0%
9127,6013367200,Contra Costa,72.2%


In [23]:
unique_values = merged_homeowners['housing_units_with_mortgage'].unique()
unique_values

array(['67.5%', '67.0%', '68.7%', '67.1%', '70.3%', '63.3%', '68.2%',
       '64.8%', '67.3%', '72.2%', '69.0%', '68.6%', '69.8%', '67.2%',
       '70.2%', '64.5%', nan, '68.8%', '64.0%', '65.2%', '66.4%', '65.6%',
       '67.4%', '73.4%', '65.4%', '63.4%', '56.2%', '52.6%', '69.7%',
       '63.2%', '61.6%', '68.1%', '63.0%', '57.2%', '56.4%', '66.2%'],
      dtype=object)

In [22]:
# replace 'N' entry with nan
merged_homeowners['housing_units_with_mortgage'].replace('N', np.nan, inplace=True)

In [25]:
# checking counties that werent included in the dataset showed up as NaN
alpine = merged_homeowners[merged_homeowners['county']=='Amador']
alpine

,census_tract,county,housing_units_with_morgage
1342,6005000201,Amador,NaN
1343,6005000202,Amador,NaN
8393,6005000500,Amador,NaN
8394,6005000401,Amador,NaN
8395,6005000301,Amador,NaN
8396,6005000402,Amador,NaN
8995,6005000102,Amador,NaN
8996,6005000303,Amador,NaN
8997,6005000101,Amador,NaN
8998,6005000304,Amador,NaN


In [26]:
merged_homeowners.to_csv('governance_morgage_metric.csv')

## Function Call

In [36]:
@append_metadata
def governance_mortgage_upload(input_csv, export=False, varname=""):  
    '''
    The function uploads a CSV file of the percentage of homeowners with mortgages to S3. Data was sources from the American Community Survey (ACS). Data code is: DP05

    Parameters
    ----------
    input_csv: str
        calculated metric csv
    export: bool, optional
        True to upload csv to AWS, False otherwise.

    Methods
    --------
    Relevant columns for the Cal-CRAI metric were isolated from the original dataset.
    Entries within rows were converted to columns for better metric entry/visualization.
    Data entries were renamed for better transparency.
    Cal-CRAI tracts were merged in with the county level data, assiging mortage percentages to each tract within its repective county.
    Missing data is kept as is (i.e., "NaN") to avoid misrepresentation by setting to zero.

    Script
    ------
    governance_homeowners_insurance.ipynb

    Note
    ------
    This function assumes users have configured the AWS CLI such that their access key / 
    secret key pair are stored in ~/.aws/credentials. 
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    ''' 
    print('Data transformation: eliminate excess headers and columns not relevant to metric calculation.')
    print('Data transformation: rename and adjust column entires to maintain cleaning standardization.')
    print('Data transformation: flatten data so metric variables become columns.')
    print('Data transformation: assign non-percentage values as nan.')

    if export == True:
        # pull csv from aws
        bucket_name = 'ca-climate-index'
        upload_csv_aws([input_csv], bucket_name, '3_fair_data/index_data')

    if export == False:
        print(f'{input_csv} uploaded to AWS.')

       # os.remove(input_csv)  # Remove local file after upload

In [38]:
input_csv = 'governance_morgage_metric.csv'
variable = 'governance_acs_homeowners_insurance'

governance_mortgage_upload(input_csv, export=False, varname='test')